### Installation des dépendances

In [ ]:
%pip install numpy
%pip install opencv-python
%pip install -U matplotlib
%pip install -U scikit-image
import numpy as np
import cv2 as cv
import math
import matplotlib.pyplot as py
from numpy import linalg as LA
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
import os
import time

# Transformée de KL

### Lecture des fichiers images

In [87]:
def plotImage(image, type):
    figure = py.figure(figsize = (10,10))
    imageout = np.clip(image,0,255)
    imageout= imageout.astype(type)
    py.imshow(imageout, cmap='gray')
    py.show()

In [152]:

# Read image
dataPath = "/data/"
currDirectory = os.getcwd()
fileList = os.listdir(currDirectory + dataPath)
quantificateur = [[8,8,8], [8,8,4], [8,8,0], [8,4,0]]
qualiteCompressionRGB = []
qualiteCompressionYUV = []

for file in fileList:
    imagePath = currDirectory + dataPath + file
    img = py.imread(imagePath)
    
    if img is not None:
        YUVimage = cv.cvtColor(img, cv.COLOR_RGB2YUV)
        for q in quantificateur:
            imageKLTransforme = compression(img, q)
            imageYUVKLTransforme = compression(YUVimage, q)
            qualiteCompressionRGB.append(qualiteCompression(img, imageKLTransforme))
            qualiteCompressionYUV.append(qualiteCompression(YUVimage, imageYUVKLTransforme))
            
        print("qualité de compression RGB : pour l'image " + file, qualiteCompressionRGB)
        qualiteCompressionRGB = []
        print("qualité de compression YUV : pour l'image " + file, qualiteCompressionYUV)
        qualiteCompressionYUV = []
    
   

C:\Users\loicn\AppData\Local\Temp\ipykernel_34632\1820860134.py:6: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  psnr = peak_signal_noise_ratio(imageOriginale, imageCompressee)


qualité de compression RGB : pour l'imagekodim01.png [(0.0002828962530074205, -34.491738338296884), (0.11435641048396585, -5.5059847563923325), (0.00268578206815842, 6.811307336521038), (0.11435641048396585, -5.5059847563923325)]
qualité de compression YUV : pour l'imagekodim01.png [(0.0003113335793745662, -35.23056593144944), (0.13131838122635597, -6.944249649604983), (0.0003275455276684391, 6.726684777875798), (0.13131838122635597, -6.944249649604983)]
qualité de compression RGB : pour l'imagekodim02.png [(0.0002690949813130911, -31.517365856052123), (0.24753711479462978, 0.10565966905394392), (0.0007578122672446558, 9.8564607294653), (0.24753711479462978, 0.10565966905394392)]
qualité de compression YUV : pour l'imagekodim02.png [(0.00034355418035111554, -34.284901701118244), (0.11864884377865281, -6.64645449355087), (0.0005330348693686247, 7.648588138979502), (0.11864884377865281, -6.64645449355087)]
qualité de compression RGB : pour l'imagekodim05.png [(0.00029422708325691014, -32

## Question 2
### Implémentation de la transformée de KL 
### Quantification et recomposition de l'image

In [88]:

def compression(image, quantificateur: list):
    # Calcul de la moyenne
    sommeR = 0.0 
    sommeG = 0.0 
    sommeB = 0.0 
    for i in range(len(image)):
        for j in range(len(image[0])):
            sommeR=sommeR+image[i][j][0]
            sommeG=sommeG+image[i][j][1]
            sommeB=sommeB+image[i][j][2]
    nbPixels = len(image)*len(image[0])        
    MoyR= sommeR / nbPixels
    MoyG= sommeG / nbPixels
    MoyB= sommeB / nbPixels
    
    # calcul de la matrice de covariance
    covRGB = np.zeros((3,3), dtype = "double")
    for i in range(len(image)):
        for j in range(len(image[0])):
            vecTemp=[[image[i][j][0] - MoyR], [image[i][j][1]] - MoyG, [image[i][j][2] - MoyB]]
            vecProdTemp = np.dot(vecTemp,np.transpose(vecTemp))
            covRGB = np.add(covRGB,vecProdTemp)
    covRGB = covRGB / nbPixels  
    
    # Calcul des valeurs propres et vecteurs propres
    eigval, eigvec = LA.eig(covRGB)
    eigvec = np.transpose(eigvec)
    
    # retirer les axes principaux
    eigvecsansAxe0 = np.copy(eigvec)
    eigvecsansAxe0[0,:] = [0.0,0.0,0.0]
    eigvecsansAxe1 = np.copy(eigvec)
    eigvecsansAxe1[1,:] = [0.0,0.0,0.0]
    eigvecsansAxe2 = np.copy(eigvec)
    eigvecsansAxe2[2,:] = [0.0,0.0,0.0]

    imageKLsansAxe0 = np.copy(image)
    imageKLsansAxe1 = np.copy(image)
    imageKLsansAxe2 = np.copy(image)

    vecMoy =[[MoyR], [MoyG], [MoyB]] 

    for i in range(len(image)):
        for j in range(len(image[0])):
            vecTemp=[[image[i][j][0]], [image[i][j][1]], [image[i][j][2]]]
            #a=Mb
            imageKLsansAxe0[i][j][:] = np.reshape(np.dot(eigvecsansAxe0,np.subtract(vecTemp,vecMoy)),(3))
            imageKLsansAxe1[i][j][:] = np.reshape(np.dot(eigvecsansAxe1,np.subtract(vecTemp,vecMoy)),(3))
            imageKLsansAxe2[i][j][:] = np.reshape(np.dot(eigvecsansAxe2,np.subtract(vecTemp,vecMoy)),(3)) 
            
    invEigvecsansAxe0 = LA.pinv(eigvecsansAxe0)
    invEigvecsansAxe1 = LA.pinv(eigvecsansAxe1)
    invEigvecsansAxe2 = LA.pinv(eigvecsansAxe2)

    vecMoy =[MoyR, MoyG, MoyB] #TODO: verifier si c'est correct à quantifier peut-être
    imageRGBsansAxe0 = np.copy(image)
    imageRGBsansAxe1 = np.copy(image)
    imageRGBsansAxe2 = np.copy(image)

    for i in range(len(image)):
        for j in range(len(image[0])):
            #b=inv(M)a
            vecTempsansAxe0=[[imageKLsansAxe0[i][j][0]], [imageKLsansAxe0[i][j][1]], [imageKLsansAxe0[i][j][2]]]
            vecTempsansAxe1=[[imageKLsansAxe1[i][j][0]], [imageKLsansAxe1[i][j][1]], [imageKLsansAxe1[i][j][2]]]
            vecTempsansAxe2=[[imageKLsansAxe2[i][j][0]], [imageKLsansAxe2[i][j][1]], [imageKLsansAxe2[i][j][2]]]     
            imageRGBsansAxe0[i][j][:] = np.add(np.reshape(np.dot(invEigvecsansAxe0,vecTempsansAxe0),(3)),vecMoy)
            imageRGBsansAxe1[i][j][:] = np.add(np.reshape(np.dot(invEigvecsansAxe1,vecTempsansAxe1),(3)),vecMoy)
            imageRGBsansAxe2[i][j][:] = np.add(np.reshape(np.dot(invEigvecsansAxe2,vecTempsansAxe2),(3)),vecMoy)

    # Quantification            
    quantisansAxe0 = imageRGBsansAxe0.max() / math.pow(2, max(quantificateur[0]-1, 1))
    quantisansAxe1 = imageRGBsansAxe1.max() / math.pow(2, max(quantificateur[1]-1, 1))
    quantisansAxe2 = imageRGBsansAxe2.max() / math.pow(2, max(quantificateur[2]-1, 1))
    
    imageRGBsansAxe0 = np.floor(imageRGBsansAxe0 / quantisansAxe0)
    imageRGBsansAxe1 = np.floor(imageRGBsansAxe1 / quantisansAxe1)
    imageRGBsansAxe2 = np.floor(imageRGBsansAxe2 / quantisansAxe2)
    
    return imageRGBsansAxe2  

In [151]:
def qualiteCompression(imageOriginale, imageCompressee):
    imageOriginale = np.dot(imageOriginale[:,:], [0.299, 0.587, 0.114])
    imageOriginale = imageOriginale.astype('double')
    imageCompressee = np.dot(imageCompressee[:,:], [0.299, 0.587, 0.114])
    imageCompressee = imageCompressee.astype('int8')
    psnr = peak_signal_noise_ratio(imageOriginale, imageCompressee)
    ssim = structural_similarity(imageOriginale, imageCompressee, data_range=img.max() - img.min())
    return ssim, psnr